In [ ]:
import openai
import pandas as pd

Define Your OpenAI Key

In [ ]:
openai_key_path = '/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/Research/nimlab/openai_key.txt'

01 - Organize Abstracts

In [ ]:
import openai
import pandas as pd

class TitleScreenAPIRevised:
    def __init__(self, api_key, csv_path):
        openai.api_key = api_key
        self.df = pd.read_csv(csv_path)
    
    def basic_word_screen(self, inclusion_keywords, exclusion_keywords):
        """
        Screen based on the presence of specific keywords in the title.
        """
        include_mask = self.df["Title"].str.contains('|'.join(inclusion_keywords), case=False, na=False)
        exclude_mask = self.df["Title"].str.contains('|'.join(exclusion_keywords), case=False, na=False)
        
        # Combine the masks: we want to include titles that have inclusion keywords and don't have exclusion keywords
        self.df["Basic_Word_Screen"] = (include_mask & ~exclude_mask).astype(int)
    
    def language_model_screen(self):
        """
        Screen titles using a basic language model approach.
        """
        keywords = ["focal", "lesion", "brain", "death", "case"]
        self.df["Language_Model_Screen"] = self.df["Title"].apply(lambda title: int(any(word in title.lower() for word in keywords)))
    
    def evaluate_with_openai(self, title, question):
        """
        Use OpenAI GPT to evaluate the title based on a posed question.
        """
        prompt = f"Title: {title}\n{question}\n\nResponse (0 for No, 1 for Yes):"
        response = openai.Completion.create(engine="davinci", prompt=prompt, max_tokens=10)
        decision_text = response.choices[0].text.strip()
        decision = 1 if "1" in decision_text else 0
        return decision
    
    def openai_screen(self, question):
        """
        Screen titles using OpenAI GPT based on a posed question.
        """
        self.df["OpenAI_Screen"] = self.df["Title"].apply(lambda title: self.evaluate_with_openai(title, question))
    
    def to_csv(self, output_path):
        """
        Save the dataframe with the screening results to a CSV.
        """
        self.df.to_csv(output_path, index=False)

# Example usage (commented out for reference):
# screener = TitleScreenAPIRevised("YOUR_OPENAI_API_KEY", "/path/to/your/csvfile.csv")
# screener.basic_word_screen(["focal", "lesion", "case"], ["review", "overview"])
# screener.language_model_screen()
# question = "Do you think this article has at least one case with a focal brain lesion causing brain death?"
# screener.openai_screen(question)
# screener.to_csv("/path/to/save/screened_csvfile.csv")


